In [52]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
from scipy.ndimage import gaussian_filter1d
import plotly.graph_objects as go


def make_agora_plot(filename: str):
    df = pd.read_csv(filename)
    df["Wind (total)"] = df["Wind offshore"] + df["Wind onshore"]

    df_melted = pd.melt(df, id_vars=["date_id"], value_vars=["Biomass", "Hydro", "Wind (total)", "Solar", "Conventional"])

    fig = px.area(df_melted, x="date_id", y="value", color="variable", line_group="variable",
        color_discrete_map={
            "Biomass": "green",
            "Hydro": "blue",
            "Solar": "yellow",
            "Conventional": "grey",
            "Wind": "lightgrey"
    })
    fig.update_layout({
        'yaxis_title':"GWh/hour",
        'xaxis_title':"Date",
        'legend_title_text':"Source"
    })

    return fig

weekly_fig = make_agora_plot("power_generation_and_consumption_week.csv")
weekly_fig.show()

yearly_fig = make_agora_plot("power_generation_and_consumption_year.csv")

# Also add a smoothed renewable share to the yearly plot

combined_yearly_fig = make_subplots(specs=[[{"secondary_y": True}]])


for trace in yearly_fig.data:
    combined_yearly_fig.add_trace(
    trace,
    secondary_y=False
)
    
yearly_df = pd.read_csv("power_generation_and_consumption_year.csv")
yearly_df["Renewable share smoothed"] = gaussian_filter1d(yearly_df['Renewable share'], sigma=4)

print(yearly_df)

renewables_share_fig = go.Scatter(x=yearly_df["date_id"], y=yearly_df["Renewable share smoothed"], 
                                  line={'color':'red'}, showlegend=False
)


combined_yearly_fig.add_trace(
    renewables_share_fig,
    secondary_y=True
)

combined_yearly_fig.update_layout(
    yaxis=dict(
        title="GWh/hour",
    ),
    yaxis2=dict(
        title="Share of renewables in %",
        titlefont=dict(
            color="red"
        ),
        tickfont=dict(
            color="red"
        )
    ),

)

combined_yearly_fig.show()


pio.write_image(weekly_fig, file="energy_production_week.pdf", format='pdf')
pio.write_image(combined_yearly_fig, file="energy_production_year.pdf", format='pdf')

                 date_id            date_id.1  Biomass  Grid emission factor  \
0    2023-01-01T00:00:00  2023-01-01T00:00:00  113.845               204.436   
1    2023-01-02T00:00:00  2023-01-02T00:00:00  116.824               325.709   
2    2023-01-03T00:00:00  2023-01-03T00:00:00  118.713               440.302   
3    2023-01-04T00:00:00  2023-01-04T00:00:00  118.295               203.504   
4    2023-01-05T00:00:00  2023-01-05T00:00:00  120.645               326.276   
..                   ...                  ...      ...                   ...   
360  2023-12-27T00:00:00  2023-12-27T00:00:00  118.750               265.032   
361  2023-12-28T00:00:00  2023-12-28T00:00:00  120.408               159.326   
362  2023-12-29T00:00:00  2023-12-29T00:00:00  121.464               151.512   
363  2023-12-30T00:00:00  2023-12-30T00:00:00  121.948               187.630   
364  2023-12-31T00:00:00  2023-12-31T00:00:00  123.518               183.094   

     Hard Coal   Hydro  Lignite  Natura